In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [7]:
?np.random.choice

Создадим сэмпл данных о чаевых в кофейне. Задаем рандомно количество посетителей в день, среднюю цену за чашку кофе и среднее количество чашек на гостя.

In [116]:
# Создаём сэмпл
n_samples = 500

customers = np.random.choice(100, n_samples) + 50
coffee_price = np.random.choice(5, n_samples) * 10 + 70
cups_per_pers = np.random.choice(4, n_samples) + 2

tips = customers * cups_per_pers * 10 + 50

data = pd.DataFrame({
    'customers': customers, 
    'cups_per_pers': cups_per_pers,
    'coffee_price': coffee_price,
    'tips': tips
})
data.head(5)

,customers,cups_per_pers,coffee_price,tips
0,146,3,80,4430
1,59,4,90,2410
2,97,5,110,4900
3,65,3,110,2000
4,107,3,90,3260


In [117]:
data.tips.median()

3140.0

Предположим, что сумма чаевых за день может зависеть от количества постителей в день.

In [121]:
from sklearn.metrics import mean_absolute_error

X = data[['customers']]
y = data['tips']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['customers']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [35.89666188]
Bias: -72.15554837099899
Error: 1008.731846387945


Ошибка 1009 достаточно велика для медианы в 3140. 
Добавим в модель к количеству посетителей среднюю цену за чашку кофе.

In [118]:
X = data[['customers', 'coffee_price']]
y = data['tips']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['customers', 'coffee_price']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [35.86567784  2.62304144]
Bias: -302.722882205866
Error: 1007.4608895721263


Ошибка 1007 по-прежнему достаточно высока.
Попробуем посчитать, сколько всего было продано чашек кофе и подставим эти данные в модель.

In [119]:
# Создаем новый признак
data['sold_cups'] = data['customers'] * data['cups_per_pers']
data.head(5)

,customers,cups_per_pers,coffee_price,tips,sold_cups
0,146,3,80,4430,438
1,59,4,90,2410,236
2,97,5,110,4900,485
3,65,3,110,2000,195
4,107,3,90,3260,321


In [120]:
X = data[['sold_cups']]
y = data['tips']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['sold_cups']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [10.]
Bias: 50.000000000000455
Error: 5.2295945351943376e-14


Очень низкая ошибка говорит о точности модели с предиктором "Количество проданных чашек кофе"